# Imports and settings

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,8

In [ ]:
import numpy as np
import pandas as pd

# Load mock Data

In [ ]:
date = [1388534400000, 1391212800000, 1393632000000, 1396310400000, 1398902400000, 1401580800000, 1404172800000, 1406851200000, 1409529600000, 1412121600000, 1414800000000, 1417392000000] #eerste dag van de maand in 2014
y = [93906.1, 74794.7, 61266.9, 24086.9, 20694.7, 1321.7, 0, 0, 0, 23711.2, 53668.7, 148793.7] #gas in kWh
x1 = [321, 277, 228, 121, 99, 26, 6, 35, 23, 92, 224, 376] #graaddagen
x2 = [21, 20, 22, 21, 19, 21, 22, 20, 22, 22, 20, 21] #weekdagen
x3 = [10, 8, 9, 9, 12, 9, 9, 11, 8, 9, 10, 10] #weekenddagen
x4 = [-20.3, -6.4, -29.3, -3.6, 0, 0, 0, 0, 0, 0, -7.3, -125.3] #vorst

In [ ]:
#convert unix timestamp to datetimeIndex
date = [x/1000 for x in date]
date = np.array(date).astype('datetime64[s]')
date = pd.DatetimeIndex(date)

In [ ]:
dict = {
    'gas': pd.Series(y, index=date),
    'graaddagen': pd.Series(x1, index=date),
    'weekdagen': pd.Series(x2, index=date),
    'weekenddagen': pd.Series(x3, index=date),
    'vorst': pd.Series(x4, index=date)
}

In [ ]:
df = pd.DataFrame(dict)

In [ ]:
for column in df.columns:
    plt.figure()
    df[column].plot(title='{}'.format(column))

# Run Test

## First with all variables

In [ ]:
from opengrid.library import multiVariateLinearRegression

In [ ]:
MVLR = multiVariateLinearRegression.MultiVariateLinearRegression(df, dependent_variable='gas')

In [ ]:
MVLR.result.params

In [ ]:
MVLR.result.summary()

The P-value for week days and weekend days is pretty high, degree days and frost seem to be statistically significant

## Repeat with Degreedays and Frost only

In [ ]:
MVLR.get_significant_variables()

In [ ]:
D2 = MVLR.get_ols_with_significant_variables()

In [ ]:
D2.summary()

# Plot Models

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot_date(df.index, df['gas'],'-',label='gas')
ax1.plot_date(df.index, MVLR.result.fittedvalues, '-', label='model with 4 variables')
ax1.plot_date(df.index, D2.fittedvalues, '-', label='model with only degreedays & frost')

plt.legend()